In [15]:
# Extract CZ01-CZ16 weather files from ZIP archives for specific year
# The ZIP archives can be downloaded from https://www.calmac.org/weather.asp

# For example, the input will be a ZIP archive like this:
# CZ2022_ALL_HIST.zip
# +---CA_EUREKA_725940_22.zip
# |   +---CA_EUREKA_725940_22.EPW
# |   +---...
# +---CA_NAPA-CO_724955_22.zip
# |   +---CA_NAPA-CO_724955_22.EPW
# |   +---...
# +---...

# The output will be a folder like this:
# weatherCY2022
# +---CZ01-CA_EUREKA_725940_CY2022.epw
# +---CZ02-CA_NAPA-CO_724955_CY2022.epw


In [16]:
from zipfile import ZipFile
from pathlib import Path
import re

In [17]:
import requests

In [18]:
from datetime import datetime

In [ ]:
# Define URLs to get CALMAC weather data for CZ2022 and CZ2025
# Each ZIP file is a nested zip archive with nested zip files containing epw, FIN4, and BINM files
DEER2024_bundle_filename = Path('CZ2022_ALL.zip')
DEER2024_bundle_href = 'https://www.calmac.org/weather/CZ2022/CZ2022_ALL.zip'

CZ2025_bundle_filename = Path('CZ2025_ALL.zip')
CZ2025_bundle_href = 'https://www.calmac.org/weather/CZ2025/CZ2025_ALL.zip'


In [20]:
# Get all CZ2025 weather files if not already present in current folder
if not CZ2025_bundle_filename.exists():
    print("Downloading", CZ2025_bundle_href)
    r0=requests.head(CZ2025_bundle_href)
    print(r0.headers)
    r1 = requests.get(CZ2025_bundle_href)
    r1.raise_for_status()
    CZ2025_bundle_filename.write_bytes(r1.content)

filestats = CZ2025_bundle_filename.stat()
print(CZ2025_bundle_filename,
      filestats.st_size,
      datetime.fromtimestamp(filestats.st_mtime).isoformat())

CZ2025_ALL.zip 60952600 2026-02-27T13:42:20.777730


In [ ]:
# Define input zip file as CZ2025 file
inputzip = CZ2025_bundle_filename

In [ ]:
# Defining the exact file name of the station in each climate zone

# DEER 2024 mapping is based on weather files used in the initial release of DEER 2024
cec_cz_map_deer2024 = [
    ('CZ01', 'CA_EUREKA_725940'),
    ('CZ02', 'CA_NAPA-CO_724955'),
    ('CZ03', 'CA_OAKLAND-METRO-AP_724930'),
    ('CZ04', 'CA_SAN-JOSE-IAP_724945'),
    ('CZ05', 'CA_SANTA-MARIA-PUBLIC-AP_723940'),
    ('CZ06', 'CA_LOS-ANGELES-IAP_722950'),
    ('CZ07', 'CA_SAN-DIEGO-LINDBERGH-F.*_722900'),
    ('CZ08', 'CA_LONG-BEACH-DAUGHERTY-FLD_722970'),
    ('CZ09', 'CA_LOS-ANGELES-DOWNTOWN-USC_722874'),
    ('CZ10', 'CA_RIVERSIDE-MUNI_722869'),
    ('CZ11', 'CA_RED-BLUFF-MUNI-AP_725910'),
    ('CZ12', 'CA_STOCKTON-METRO-AP_724920'),
    ('CZ13', 'CA_FRESNO-YOSEMITE-IAP_723890'),
    ('CZ14', 'CA_DAGGETT-BARSTOW-.*_723815'),
    ('CZ15', 'CA_EL-CENTRO-NAF_722810'),
    ('CZ16', 'CA_BISHOP-AP_724800'),
]

# CZ2025 mapping is based on the Title 24 Energy Code Accounting Report (2024)
cec_cz_map_cz2025 = [
    ('CZ01', 'CA_ARCATA-AP_725945',725945),
    ('CZ02', 'CA_SANTA-ROSA(AWOS)_724957',724957),
    ('CZ03', 'CA_OAKLAND-METRO-AP_724930',724930),
    ('CZ04', 'CA_PASO-ROBLES-MUNI-AP_723965',723965),
    ('CZ05', 'CA_SANTA-MARIA-PUBLIC-AP_723940',723940),
    ('CZ06', 'CA_LOS-ANGELES-IAP_722950',722950),
    ('CZ07', 'CA_SAN-DIEGO-LINDBERGH-FIELD_722900',722900),
    ('CZ08', 'CA_FULLERTON-MUNI-AP_722976',722976),
    ('CZ09', 'CA_BURBANK-GLNDLE-PASAD-AP_722880',722880),
    ('CZ10', 'CA_RIVERSIDE-MUNI_722869',722869),
    ('CZ11', 'CA_RED-BLUFF-MUNI-AP_725910',725910),
    ('CZ12', 'CA_SACRAMENTO-EXECUTIVE-AP_724830',724830),
    ('CZ13', 'CA_FRESNO-YOSEMITE-IAP_723890',723890),
    ('CZ14', 'CA_PALMDALE-AP_723820',723820),
    ('CZ15', 'CA_PALM-SPRINGS-IAP_722868',722868),
    ('CZ16', 'CA_BLUE-CANYON-AP_725845',725845),
]

In [43]:
# Defining a function to get the list of all files in the nested zip archive
# Returns a list of tuples (zi1, zi2) where zi1 is the name of the zip file for a given station
# and zi2 is any file within that zip (epw, FIN4, or BINM)
def get_contents(inputzip: Path) -> list:
    contents=[]
    with ZipFile(inputzip) as zf1:
        for zi1 in zf1.infolist():
            print(zi1.filename)
            try:
                with ZipFile(zf1.open(zi1)) as zf2:
                    for zi2 in zf2.infolist():
                        print("+---",zi2.filename)
                        contents.append((zi1.filename,zi2.filename))
            except:
                pass
    return contents

# Defining a function to get the relevant epw files based on defined CZ map
# Returns a tuple (zi1, zi2) where zi1 is the name of the zip file for a given station
# and zi2 is the name of the epw file within that zip
def get_cz_file(station: str, contents: list) -> tuple:
        print("Searching for", station)
        for fnames in contents:
            # criteria 1. find the matching weather station.
            # criteria 2. Find the EPW file within.
            if (
                fnames[0].startswith(station)
                and fnames[-1].upper().endswith('EPW')
            ):
                print(fnames)
                return(fnames)
        raise RuntimeError(f"No EPW file found for station {station}")

# Defining a function to extract relevant epw files and save them in the output folder
# The output file name will be in the format of CZxx-CALMACstation.epw, where xx is the climate zone number and CALMACstation is the file name as saved on CALMAC
def extract_weather(cec_cz_map: list, contents: list, inputzip: Path, output_folder: Path):
    output_folder.mkdir(exist_ok=True)
    for (cz, station, station_id) in cec_cz_map:
        zi1, zi2 = get_cz_file(station, contents)
        output_file = output_folder / (cz + '-' + zi2)
        
        with ZipFile(inputzip) as zf1:
            with ZipFile(zf1.open(zi1)) as zf2:
                with zf2.open(zi2) as file_epw:
                    output_file.write_bytes(file_epw.read())
                    print(output_file)

In [ ]:
# Define output folder and run the extraction function
output_folder = Path('CZ2025')
contents = get_contents(inputzip)
print()
extract_weather(cec_cz_map_cz2025, contents, inputzip, output_folder)
print('Done')

CA_ALTURAS_725958_CZ2025.zip
+--- CA_ALTURAS_725958_CZ2025.BINM
+--- CA_ALTURAS_725958_CZ2025.epw
+--- CA_ALTURAS_725958_CZ2025.FIN4
+--- Terms of Use.pdf
+--- Importing FIN4 files into Excel.txt
CA_ARCATA-AP_725945_CZ2025.zip
+--- CA_ARCATA-AP_725945_CZ2025.BINM
+--- CA_ARCATA-AP_725945_CZ2025.epw
+--- CA_ARCATA-AP_725945_CZ2025.FIN4
+--- Terms of Use.pdf
+--- Importing FIN4 files into Excel.txt
CA_AUBURN-MUNI-AP_720267_CZ2025.zip
+--- CA_AUBURN-MUNI-AP_720267_CZ2025.BINM
+--- CA_AUBURN-MUNI-AP_720267_CZ2025.epw
+--- CA_AUBURN-MUNI-AP_720267_CZ2025.FIN4
+--- Terms of Use.pdf
+--- Importing FIN4 files into Excel.txt
CA_BAKERSFIELD-MEADOWS-FLD_723840_CZ2025.zip
+--- CA_BAKERSFIELD-MEADOWS-FLD_723840_CZ2025.BINM
+--- CA_BAKERSFIELD-MEADOWS-FLD_723840_CZ2025.epw
+--- CA_BAKERSFIELD-MEADOWS-FLD_723840_CZ2025.FIN4
+--- Terms of Use.pdf
+--- Importing FIN4 files into Excel.txt
CA_BEALE-AFB_724837_CZ2025.zip
+--- CA_BEALE-AFB_724837_CZ2025.BINM
+--- CA_BEALE-AFB_724837_CZ2025.epw
+--- CA_BEAL

In [ ]:
# Define One Building URLs corresponding to relevant CALMAC stations for extracting ddy files
URLS = [
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ01_Arcata-Eureka-California.Redwood.Coast-Humboldt.County.AP.725945_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ02_Santa.Rosa-Schulz-Sonoma.County.AP.724957_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ03_Oakland.Intl.AP.724930_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ04_Paso.Robles.Muni.AP.723965_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ05_Santa.Maria.Public.AP-Hancock.Field.723940_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ06_Los.Angeles.Intl.AP.722950_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ07_San.Diego.Intl.AP-Lindbergh.Field.722900_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ08_Fullerton.Muni.AP.722976_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ09_Hollywood.Burbank.AP.722880_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ10_Riverside.Muni.AP.722869_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ11_Red.Bluff.Muni.AP.725910_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ12_Sacramento.Exec.AP.724830_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ13_Fresno.Yosemite.Intl.AP.723890_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ14_Palmdale.Rgnl.AP.723820_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ15_Palm.Springs.Intl.AP.722868_CTZ2025.zip",
    "https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/California_Climate_Zones/California_CTZ_2025/USA_CA_CZ16_Blue.Canyon-Nyack.AP.725845_CTZ2025.zip",
]

# Define output folder
output_folder = Path("CZ2025")
output_folder.mkdir(parents=True, exist_ok=True)

In [ ]:
# Pattern matching CZ in the ddy filename, and mapping to the corresponding epw filename
CZ_RE = re.compile(r"(CZ\d{2})", re.IGNORECASE)

cz_to_epw = {}

for epw in output_folder.glob("*.epw"):
    m = CZ_RE.search(epw.name)
    if m:
        cz_to_epw[m.group(1).upper()] = epw.name

if not cz_to_epw:
    raise RuntimeError("No .epw files with CZ## found in output_folder")

In [ ]:
# Loop through the One Building URLs, download the ZIP files, extract .ddy files, and save them with the correct names
for url in URLS:
    zip_name = Path(url).name
    zip_path = output_folder / zip_name

    # Download the ZIP file if it doesn't exist
    if not zip_path.exists():
        print("Downloading", url)

        r0 = requests.head(url, allow_redirects=True)
        print(r0.headers)

        r1 = requests.get(url)
        r1.raise_for_status()
        zip_path.write_bytes(r1.content)

    filestats = zip_path.stat()
    print(
        zip_path,
        filestats.st_size,
        datetime.fromtimestamp(filestats.st_mtime).isoformat()
    )

    # Extract .ddy files from the ZIP archive and save them with the correct names
    with ZipFile(zip_path, "r") as z:
        ddy_members = [n for n in z.namelist() if n.lower().endswith(".ddy")]
        if not ddy_members:
            raise RuntimeError(f"No .ddy file in {zip_name}")

        for member in ddy_members:
            member_name = Path(member).name
            m = CZ_RE.search(member_name) or CZ_RE.search(zip_name)
            if not m:
                raise RuntimeError(f"Cannot determine CZ for {member_name}")

            cz = m.group(1).upper()

            if cz not in cz_to_epw:
                raise RuntimeError(f"No matching EPW file for {cz}")

            epw_name = cz_to_epw[cz]
            ddy_name = Path(epw_name).with_suffix(".ddy").name
            ddy_path = output_folder / ddy_name

            with z.open(member) as src, open(ddy_path, "wb") as dst:
                dst.write(src.read())

            print(f"Extracted {member_name} -> {ddy_name}")

CZ2025\USA_CA_CZ01_Arcata-Eureka-California.Redwood.Coast-Humboldt.County.AP.725945_CTZ2025.zip 397482 2026-02-27T13:44:10.587627
Extracted USA_CA_CZ01_Arcata-Eureka-California.Redwood.Coast-Humboldt.County.AP.725945_CTZ2025.ddy -> CZ01-CA_ARCATA-AP_725945_CZ2025.ddy
CZ2025\USA_CA_CZ02_Santa.Rosa-Schulz-Sonoma.County.AP.724957_CTZ2025.zip 399366 2026-02-27T13:44:57.975804
Extracted USA_CA_CZ02_Santa.Rosa-Schulz-Sonoma.County.AP.724957_CTZ2025.ddy -> CZ02-CA_SANTA-ROSA(AWOS)_724957_CZ2025.ddy
CZ2025\USA_CA_CZ03_Oakland.Intl.AP.724930_CTZ2025.zip 397624 2026-02-27T13:44:59.166113
Extracted USA_CA_CZ03_Oakland.Intl.AP.724930_CTZ2025.ddy -> CZ03-CA_OAKLAND-METRO-AP_724930_CZ2025.ddy
CZ2025\USA_CA_CZ04_Paso.Robles.Muni.AP.723965_CTZ2025.zip 408894 2026-02-27T13:45:00.335091
Extracted USA_CA_CZ04_Paso.Robles.Muni.AP.723965_CTZ2025.ddy -> CZ04-CA_PASO-ROBLES-MUNI-AP_723965_CZ2025.ddy
CZ2025\USA_CA_CZ05_Santa.Maria.Public.AP-Hancock.Field.723940_CTZ2025.zip 402947 2026-02-27T13:45:01.514046
Ex